In [1]:
from nnunetv2.paths import nnUNet_results, nnUNet_raw
import torch
from batchgenerators.utilities.file_and_folder_operations import join
from nnunetv2.inference.predict_from_raw_data import nnUNetPredictor
from nnunetv2.imageio.mrc_reader_writer import MRCIO
import numpy as np
import mrcfile

In [2]:
def save_tomo(data, path, voxel_size=17.14):
    """
    Save a 3D numpy array as an MRC file.

    Parameters:
    - data: ndarray
        The 3D data to save.
    - path: str
        Path where the MRC file will be saved.
    - voxel_size: float
        The voxel size of the data.
    """
    with mrcfile.new(path, overwrite=True) as mrc:
        data = data.astype(np.float32)
        mrc.set_data(data)
        mrc.voxel_size = voxel_size

In [3]:
# instantiate the nnUNetPredictor
predictor = nnUNetPredictor(
    tile_step_size=0.5,
    use_gaussian=True,
    use_mirroring=True,
    perform_everything_on_device=True,
    device=torch.device('cuda', 0),
    verbose=False,
    verbose_preprocessing=False,
    allow_tqdm=True
)
# # initializes the network architecture, loads the checkpoint
# predictor.initialize_from_trained_model_folder(
#     join(nnUNet_results, 'Dataset003_3tomo/nnUNetTrainer__nnUNetPlans__3d_fullres'),
#     use_folds=(1,),
#     checkpoint_name='checkpoint_best.pth',
# )

# initializes the network architecture, loads the checkpoint
predictor.initialize_from_trained_model_folder(
    # join(nnUNet_results, 'Synapse256_nnUNet/nnUNetTrainer__nnUNetPlans__3d_fullres'),
    join(nnUNet_results, 'Dataset007_1tomo_11classes/nnUNetTrainer__nnUNetPlans__3d_fullres'),
    use_folds=(0,),
    checkpoint_name='checkpoint_best.pth',
)

In [ ]:
# img, props = MRCIO().read_images([join(nnUNet_raw, 'Synapse_128/imagesTs/pp1033.mrc')])
img, props = MRCIO().read_images(['/media/liushuo/data1/data/synapse_seg/pp0365/pp0365-bin8-wbp_1714.mrc'])
# ret = predictor.predict_single_npy_array(img, props, None, None, False)
ret1, ret2 = predictor.predict_single_npy_array(img, props, None, None, True)

100%|██████████| 1134/1134 [05:32<00:00,  3.41it/s]


Prediction on device was unsuccessful, probably due to a lack of memory. Moving results arrays to CPU


 72%|███████▏  | 821/1134 [04:05<01:30,  3.45it/s]

In [ ]:
save_tomo(ret1, f'/media/liushuo/data1/data/synapse_seg/pp0365/pp0365-bin8-wbp_corrected_ret1_11classes.mrc')
# save_tomo(ret2, f'/media/liushuo/data1/data/actin trace/actintomo/20200820/tomo/pp1776/ret2.mrc')

In [11]:
predictor.predict_from_files([['/home/liushuo/Documents/data/stack-out_demo/p2/ves_seg/p2_wbp_corrected.mrc']],
                                 [['/home/liushuo/Documents/data/stack-out_demo/p2/ves_seg/p2_output_best.mrc']],
                                 save_probabilities=False, overwrite=True,
                                 num_processes_preprocessing=2, num_processes_segmentation_export=2,
                                 folder_with_segs_from_prev_stage=None, num_parts=1, part_id=0)

TypeError: expected str, bytes or os.PathLike object, not list